# ЗАДАНИЕ
Проанализируйте результаты эксперимента и напишите свои рекомендации менеджеру.

Mobile Games AB Testing with Cookie Cats

Данная промежуточная аттестация оценивается по системе "зачет" / "не зачет".
"Зачет" ставится, если Слушатель успешно выполнил задание.
"Незачет" ставится, если Слушатель не выполнил задание.

Критерии оценивания: 1 - Слушатель проанализировал результат эксперимента

In [ ]:
# загружаем необходимые библиотеки (а по факту все, что были на семинаре)
from scipy import stats
import numpy as np

import warnings
warnings.filterwarnings('ignore')
warnings.warn('DelftStack')
warnings.warn('Do not show this message')

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [ ]:
# загружаем файл и смотрим на таблицу в первом приближении
data = pd.read_excel('результаты А_B.xlsx')
data.head(10)

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,0,0
1,337,gate_30,38,1,0
2,377,gate_40,165,1,0
3,483,gate_40,1,0,0
4,488,gate_40,179,1,1
5,540,gate_40,187,1,1
6,1066,gate_30,0,0,0
7,1444,gate_40,2,0,0
8,1574,gate_40,108,1,1
9,1587,gate_40,153,1,0


In [ ]:
# проверим поле userid на уникальность:
data["userid"].nunique()

90189

In [ ]:
# смотрим сводную информацию
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  int64 
 4   retention_7     90189 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 3.4+ MB


Как видно из сводной инфы: целостность таблицы не нарушена - все столбцы заполнены по всем 90 189 строкам, типы данных корректные. Тип данных object, возможно избыточен (немного почитал об этом), но думаю на выполнение текущей задачи это не повлияет.

In [ ]:
# ну и еще одна проверка
data.describe()

,userid,sum_gamerounds,retention_1,retention_7
count,9.018900e+04,90189.000000,90189.000000,90189.000000
mean,4.998412e+06,51.872457,0.445210,0.186065
std,2.883286e+06,195.050858,0.496992,0.389161
min,1.160000e+02,0.000000,0.000000,0.000000
25%,2.512230e+06,5.000000,0.000000,0.000000
50%,4.995815e+06,16.000000,0.000000,0.000000
75%,7.496452e+06,51.000000,1.000000,0.000000
max,9.999861e+06,49854.000000,1.000000,1.000000


Посмотрим на две группы раздельно:

In [ ]:
data[data['version']=='gate_30'].describe()

,userid,sum_gamerounds,retention_1,retention_7
count,4.470000e+04,44700.000000,44700.000000,44700.000000
mean,4.987564e+06,52.456264,0.448188,0.190201
std,2.881026e+06,256.716423,0.497314,0.392464
min,1.160000e+02,0.000000,0.000000,0.000000
25%,2.505469e+06,5.000000,0.000000,0.000000
50%,4.983631e+06,17.000000,0.000000,0.000000
75%,7.481497e+06,50.000000,1.000000,0.000000
max,9.999710e+06,49854.000000,1.000000,1.000000


In [ ]:
data[data['version']=='gate_40'].describe()

,userid,sum_gamerounds,retention_1,retention_7
count,4.548900e+04,45489.000000,45489.000000,45489.000000
mean,5.009073e+06,51.298776,0.442283,0.182000
std,2.885496e+06,103.294416,0.496663,0.385849
min,3.770000e+02,0.000000,0.000000,0.000000
25%,2.517171e+06,5.000000,0.000000,0.000000
50%,5.007329e+06,16.000000,0.000000,0.000000
75%,7.510762e+06,52.000000,1.000000,0.000000
max,9.999861e+06,2640.000000,1.000000,1.000000


Рассчитаем коэффициенты удержания между двумя группами A/B:

In [ ]:
df_retention = data[["retention_1","retention_7"]].mean()*100
print(f'Коэффициент 1-дневного удержания: {round(df_retention[0],2)}%')
print(f'Коэффициент 7-дневного удержания: {round(df_retention[1],2)}%')

Коэффициент 1-дневного удержания: 44.52%
Коэффициент 7-дневного удержания: 18.61%


In [ ]:
# расчет 1-дневного и 7-дневного удержания для каждой группы A/B:
df_retention_ab = data.groupby("version").agg({"userid":"count", "retention_1":"mean","retention_7":"mean", "sum_gamerounds":"sum"})
df_retention_ab

,userid,retention_1,retention_7,sum_gamerounds
version,,,,
gate_30,44700,0.448188,0.190201,2344795
gate_40,45489,0.442283,0.182000,2333530


При сравнении двух групп видно, что показатели ухудшаются при gate_40. Бутстрап в примере тов. иностранца конечно решает мой вопрос (побыстрее доделать задачу курса, параллельно продолжая думать "а оно мне пригодится по работе или нет"), но попробую как в прошлой домашке.

In [ ]:
# сравним показатели 1-дневного и 7-дневного удержания в разных группах. Начнем с "retention_1":
k1_retention_1 = 0.448188
k2_retention_1 = 0.442283
n1 = 44700
n2 = 45489
from statsmodels.stats import proportion
chisq, pvalue, table = proportion.proportions_chisquare(np.array([k1_retention_1, k2_retention_1]),
                                                   np.array([n1, n2]))

print('Results are ','chisq =%.3f, pvalue = %.3f'%(chisq, pvalue))

Results are  chisq =0.000, pvalue = 0.988


In [ ]:
if abs(pvalue) < 0.05:
    print("Мы можем отвергнуть нулевую гипотезу!")
else:
    print("Нам не удалось отвергнуть нулевую гипотезу")

Нам не удалось отвергнуть нулевую гипотезу


Проверка на хи-квадрат показала, что статразличия между двумя группами есть. Группа с "gate_30" предпочтительнее.

In [ ]:
# теперь "retention_7":
k1_retention_7 = 0.190201
k2_retention_7 = 0.182000
chisq, pvalue, table = proportion.proportions_chisquare(np.array([k1_retention_7, k2_retention_7]),
                                                   np.array([n1, n2]))

print('Results are ','chisq =%.3f, pvalue = %.3f'%(chisq, pvalue))

Results are  chisq =0.000, pvalue = 0.985


In [ ]:
if abs(pvalue) < 0.05:
    print("Мы можем отвергнуть нулевую гипотезу!")
else:
    print("Нам не удалось отвергнуть нулевую гипотезу")

Нам не удалось отвергнуть нулевую гипотезу


Вторая проверка тоже показывает преимущество группы "gate_30".